# **15 Puzzle**

### 15 Puzzle is a sliding puzzle.

## Libraries Used:
- queue
- time
- os
- psutil
- sys
- copy




In [3]:
#from queue import Queue
#import time
#import os
#import psutil
#import sys
#import copy

from helperfunctions import *

Goal matrix looks like this:

<img src="images/goalmatrix.jpg" width="200">

The 0 tile is the only tile that can be slid

In [4]:
goal_matrix = [
  [1, 2, 3, 4],
  [5, 6, 7, 8],
  [9, 10, 11, 12],
  [13, 14, 15, 0]
]

Input files are provided in the input folder. Feel free to switch between input files by changing the value of `input_file`

In [1]:
input_file = 'input.txt'

In [5]:
f = open('input/' + input_file)
line = f.readline()
results = line.split(', ')
arr = [int(i) for i in results]
f.close()

matrix = get_matrix(arr)  
print_matrix(matrix)

[1, 0, 2, 4]
[5, 7, 3, 8]
[9, 6, 11, 12]
[13, 10, 14, 15]



Node class for states of puzzle

Contents:
  matrix: state of puzzle
  parent: state of puzzle that the node is derived from
  move: move made by parent node to get to state
  g_n:
  h_n:
  f_n


## Breadth First Search

In [6]:
"""
  Function to solve 15 puzzle using bfs.
    
  Arguments:
    4 by 4 matrix
  Returns:
    goal state node (unless program exits due to an empty queue)
"""
def bfs(matrix): 
  # start timer 
  initial_time = time.process_time()
  
  process = psutil.Process(os.getpid())
  initial_memory = process.memory_info().rss / 1024.000000
  
  expanded_nodes = []
  expanded_nodes_count = 0
  
  # head node
  initial_node = Node(matrix) 

  # check to see if initial matrix is the goal state
  # if so, return that node
  if (initial_node.matrix == goal_matrix):
    return initial_node
  
    
  # initialize queue and enqueue head node
  q = Queue()
  q.put(initial_node)  

  while True:
    # if queue is empty, no solution is able to be found using this program
    if(q.empty() == True):
      sys.exit('BFS failed to find a solution.')
        
    # get node at the front of queue
    node = q.get()
        
    # if node has not been expanded yet, add it to expanded nodes
    if (expanded_nodes.__contains__(node.matrix) == False):
      expanded_nodes.append(node.matrix)
      expanded_nodes_count += 1
        
    children = get_children(node, expanded_nodes)

    for child in children:
      if (child.matrix == goal_matrix):
        # stop time when goal matrix is found
        elapsed_time = time.process_time() - initial_time
        # get memory usage
        final_memory = process.memory_info().rss / 1024.000000
        memory_used = final_memory - initial_memory
        print_search_info(child, expanded_nodes_count, elapsed_time, memory_used)
        return False
        #return child
      if (child.matrix != None):
        q.put(child)

In [7]:
# list for expanded nodes and list for moves


print('\n**15 Puzzle using BFS**\n')    
    
print('\ninitial state:')
print_matrix(matrix)


node = bfs(matrix)



**15 Puzzle using BFS**


initial state:
[1, 0, 2, 4]
[5, 7, 3, 8]
[9, 6, 11, 12]
[13, 10, 14, 15]

Moves:  ['R', 'D', 'L', 'D', 'D', 'R', 'R']
Number of Nodes  Expanded:  152
Time Taken:  0.0312500 nanoseconds
Memory Used:  172.00 KB



## Iterative Deepening Depth First Search using Depth Limited Search

In [ ]:
DLS

In [11]:

"""
    Recursive helper function to iddfs().
    Depth Limited Search function.
    Searches for goal node up until depth limit which is passed as an arg
    
    Arguments:
      node: Node object containing the matrix, move to get to the state, and the parent of the node
      depth: the depth to search to on a node
        
    Returns:
      (Node(), boolean)
      an unitialized node if goal is not found or the goal node
      True if there are children nodes, False if there are no children nodes      
        
"""
def dls(node, depth, expanded): 
  # goal node is found, return it
  if (node.matrix == goal_matrix):
    return node, True

  # depth limit reached, so return None, True 
  # bc there may be children node to search
  elif (depth == 0):
    return Node(), True
  
  # depth limit not reached and goal node not found,
  # so get children and continue dls
  else:
    # get position of blank tile
    x, y, i, j = 99, 99, 0, 0
    while (i < 4):
      j = 0
      while (j < 4):
        if (node.matrix[i][j] == 0):
          x, y = i, j
        j = j + 1
      i = i + 1
        
    nodes_remaining = False

    # get children 
    children = get_children(node, expanded)
    

    for child in children:
      # check if child exists
      if (child.matrix != None):
        # call dls for child if it exists
        found, remaining = dls(child, depth - 1, expanded)
        if (found.matrix != None):
          # goal node found, return it
          return found, True
        # goal node not found, but there are remaining nodes
        if (remaining == True):
          nodes_remaining = True
    
    # no remaining nodes => ends iddfs
    return Node(), nodes_remaining

In [ ]:
IDDFS

In [16]:
"""
    Iterative Deepening Depth First Search function.
    Searches for goal node by increasing the depth limit by 1 with each iteration.
    
    Arguments:
      node: Node object containing the initial state
        
    Returns:
      an unitialized node if goal is not found or the goal node if the goal node is reached      
        
"""
def iddfs(matrix):
  
  # start time right before call to bfs  
  initial_time = time.process_time()

  # start memory usage calculation
  process = psutil.Process(os.getpid())
  initial_memory = process.memory_info().rss / 1024.000000

  expanded_nodes_count = 0
  expanded_nodes = []
  
  node = Node(matrix)
  
  # begin at depth 0
  depth = 0
  while (depth >= 0):
    expanded_nodes_count = expanded_nodes_count + len(expanded_nodes)
    # reset visited list for next iteration of dls
    expanded_nodes = []
    expanded_nodes.append(node.matrix)
    found, remaining = dls(node, depth, expanded_nodes)
    if (found.matrix != None):
      # stop time when goal matrix is found
      elapsed_time = time.process_time() - initial_time
      # get memory usage
      final_memory = process.memory_info().rss / 1024.000000
      memory_used = final_memory - initial_memory
      print_search_info(found, expanded_nodes_count, elapsed_time, memory_used)
      return found
    # dls will set remaining to false when there are no more nodes to expand
    # function will then stop looping and return an empty node
    elif(remaining == False):
      return Node()
    depth = depth + 1
    
    
    

In [17]:
print('\n**15 Puzzle using IDDFS**\n')
   
# run iddfs
node = iddfs(matrix)



**15 Puzzle using IDDFS**


initial state:
[1, 2, 3, 4]
[5, 10, 6, 0]
[9, 7, 11, 8]
[13, 14, 15, 12]

Moves:  ['D', 'L', 'L', 'U', 'R', 'D', 'R', 'D']
Number of Nodes  Expanded:  8
Time Taken:  0.4218750 nanoseconds
Memory Used:  0.00 KB



# A*

In [5]:
"""
  Function for calculating heuristic 1 (misplaced_tiles)
"""
def h1_misplaced_tiles(matrix):
  misplaced_tiles = 0
  i, j = 0, 0
  while (i < 4):
    j = 0
    while (j < 4):
      # blank tile does not count in distance
      if (matrix[i][j] != goal_matrix[i][j] and matrix[i][j] != 0):
        misplaced_tiles = misplaced_tiles + 1 
      j = j + 1
    i = i + 1

  return misplaced_tiles


"""
  Function for calculating heuristic 2 (manhattan distance)
"""
def h2_manhattan_distance(matrix):
  i = 0
  distance = 0
  while (i < 4):
    j = 0
    while (j < 4):
      if(matrix[i][j] != goal_matrix[i][j] and matrix[i][j] != 0):
        value = matrix[i][j]
        k = 0
        while (k < 4):
          l = 0
          while (l < 4):
            if (goal_matrix[k][l] == value):
              dis = abs((i + j) - (k + l))
              distance = distance + dis
              l, k = 4, 4
            l = l + 1
          k = k + 1
      j = j + 1
    i = i + 1
  return distance

In [32]:
"""
  Function to solve 15 puzzle using a*.

  Arguments:
    4 by 4 matrix
    heuristic type
      'h1' or 'h2'
  Returns:
      goal state node (unless program exits due to an empty open list)
"""
def a_star(matrix, heuristic_type): 
  process = psutil.Process(os.getpid())
  initial_memory = process.memory_info().rss / 1024.000000

  initial_time = time.process_time()

  # head node
  initial_node = Node(matrix) 
  
  initial_node.g_n = 0
  initial_node.f_n = 0
  open_list = []
  closed_list = []

  open_list.append(initial_node)
  
  expanded_nodes_count = 0

  
  while (len(open_list) > 0):
    timed_out = time.process_time() - initial_time
    if (timed_out > 3):
      print('A* timed out')
      return
    pop_index = 0

    i = 0
    # get index of the node with the lowest f(n)
    while (i < len(open_list)):
      if (open_list[i].f_n < open_list[pop_index].f_n):
        pop_index = i
      i += 1

    # pop node with the lowest f(n)
    node = open_list.pop(pop_index)
    if(node.matrix == goal_matrix):
      # stop time and memory after bfs returns
      elapsed_time = time.process_time() - initial_time
      final_memory = process.memory_info().rss / 1024.000000
      memory_used = final_memory - initial_memory
      print_search_info(node, expanded_nodes_count, elapsed_time, memory_used)
      return node

    expanded_nodes_count += 1

    # get position of blank tile
    x, y, i, j = 99, 99, 0, 0
    while (i < 4):
      j = 0
      while (j < 4):
        if (node.matrix[i][j] == 0):
          x, y = i, j
        j = j + 1
      i = i + 1

    # get children 
    children = get_children(node, [])

    for child in children:
      if (child.matrix != None):

        # calculate f(n)
        child.g_n = node.g_n + 1
        child.h_n = None
        if (heuristic_type == 'h1'):
          child.h_n = h1_misplaced_tiles(child.matrix)
        else:
          child.h_n = h2_manhattan_distance(child.matrix)

        # calculate and save f(n) aka priority
        child.f_n = child.g_n + child.h_n

        # child already visited
        for n in closed_list:
          if (child == n):
            continue

        # child is already in open with lower g(n)
        for n in open_list:
          if (child.matrix == n.matrix and child.f_n > n.f_n):
            continue

        open_list.append(child)

    closed_list.append(node)

  # open is empty, no solution found
  print('A* failed to find a solution.')
  
  

In [33]:
print('\n15 Puzzle using A*\n')

print('A* using h1...')
node = a_star(matrix, 'h1')


print('A* using h2...')
node = a_star(matrix, 'h2')



15 Puzzle using A*

A* using h1...

Moves:  ['D', 'L', 'L', 'U', 'R', 'D', 'R', 'D']
Number of Nodes  Expanded:  37
Time Taken:  0.0156250 nanoseconds
Memory Used:  12.00 KB

A* using h2...

Moves:  ['D', 'L', 'L', 'U', 'R', 'D', 'R', 'D']
Number of Nodes  Expanded:  162
Time Taken:  0.0781250 nanoseconds
Memory Used:  164.00 KB



IDA*

In [19]:
"""
  Helper function to idastar().
  Arguments:
    path: nodes being searched
    g: step cost
    cutoff: current threshold
  Returns:
    'found' if solution is found
    new cutoff(int) if higher cutoff is needed
    float('inf') if no solution
"""
def search(path, g, cutoff, heuristic): 
  node = path[0]
  
  explored_nodes.append(node.matrix)
   
  #print_matrix(node.matrix)
  #print('\n')

  f_n = 0 
  if (heuristic == 'h1'):
    f_n = g + h1_misplaced_tiles(node.matrix)
  else:
    f_n = g + h2_manhattan_distance(node.matrix)

  if (f_n > cutoff):
    #print('cutoff reached')
    return f_n  # greater f found

  if (node.matrix == goal_matrix):
    return 'found' 

  min = float('inf')

  for child in get_children(node, []):
    # node already explored or in path, so skip it
    if child.matrix in explored_nodes:
      continue
    if child in path:
      continue

    # if child is not None, search recursively
    if (child.matrix != None):
      path.insert(0, child) 
      temp = search(path, g + 1, cutoff, heuristic)

      if (temp == 'found'):
        # solution found
        return 'found'

      if (temp < float('inf')):
        # higher cutoff found
        min = temp

      path.pop(0)

  # no solution, return infinity
  return min

In [21]:
"""
  Function to solve 15 puzzle with Iterative Deepening A*
  Arguments:
    matrix: 4 by 4, list of 4 lists
"""
def idastar(matrix, heuristic):
  
  initial_time = time.process_time()
  process = psutil.Process(os.getpid())
  initial_memory = process.memory_info().rss / 1024.000000
  
  # initialization
  path = []
  cutoff = 0
  expanded_nodes = []
  global explored_nodes
  explored_nodes = []

  # set cutoff
  if (heuristic == 'h1'):
    cutoff = h1_misplaced_tiles(matrix)
  else:
    cutoff = h2_manhattan_distance(matrix)

  root = Node(matrix)
  path.insert(0, root)

  iterations = 0

  while True:
    explored_nodes = []

    iterations += 1

    # begin searching
    temp = search(path, 0, cutoff, heuristic)

    if (temp == 'found'):
      print('\nSolved in', iterations, 'iterations using', heuristic)
      elapsed_time = time.process_time() - initial_time
      final_memory = process.memory_info().rss / 1024.000000
      memory_used = final_memory - initial_memory
      print_search_info(path[0], len(explored_nodes), elapsed_time, memory_used)
      return path[0]

    # no solution
    if (temp == float('inf')):
      print('This algorithm was not able to find a solution')
      return

    # increase cutoff
    cutoff = temp
    
  

In [22]:

print('15 Puzzle using Iterative Deepening A*')
node = idastar(matrix, 'h1')


15 Puzzle using Iterative Deepening A*

Solved in 3 iterations using h1

Moves:  ['D', 'L', 'L', 'U', 'R', 'D', 'R', 'D']
Number of Nodes  Expanded:  45
Time Taken:  0.0000000 nanoseconds
Memory Used:  0.00 KB



Lets compare all

comparisons here